<a href="https://colab.research.google.com/github/imtiaj-sreejon/Coursera_Capstone/blob/master/Capstone_Project_Notebook_for_IBM_Applied_Data_Science_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project Notebook
This notebook is for the <font color='green'>**capstone project**</font> under <font color='teal'>*IBM Applied Data Science Professional Course*</font> hosted on Coursera.

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!
